In [1]:
import pandas as pd
import fiona
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
#import dedupe
from dotenv import load_dotenv
import os
import pyArango
import usaddress
from usaddress import tag
from scourgify import normalize_address_record
import re
import json

In [2]:
from pyArango.connection import *
conn = Connection(username="root", password="0505")
corp_db = conn["corp_data"]

In [3]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [10]:
# extracting the values of the indiv_unique_address collection
indiv_aql = "FOR x IN indiv_unique_address RETURN x"
indiv_query_result = corp_db.AQLQuery(indiv_aql,rawResults=True, batchSize = 1000)
col_indiv = {}
ind_indiv = 0

for indiv in indiv_query_result:
    col_indiv[ind_indiv] = indiv
    ind_indiv += 1

indiv_df = pd.DataFrame.from_dict(data = col_indiv, orient = 'index')
indiv_ar = indiv_df.to_numpy()

In [29]:
# a little bit more preprocessing

indiv_df.loc[:,'NamePlusAddr'] = [' '.join((str(a),str(b))) for a,b in zip(indiv_df['IndividualNameShort'],indiv_df['Addr'])]
indiv_ar_2 = indiv_df.to_numpy()

unique_keys, indices = np.unique(indiv_ar_2[:,7], return_index=True)
indiv_unique = indiv_ar_2[np.sort(indices)]
# len(indiv_unique): 2092355

indiv_u = np.delete(indiv_unique,(0,1,2),1)
indiv_df_final = pd.DataFrame(indiv_u,columns = ['DataID','IndividualName','IndividualNameShort','Addr','NamePlusAddr'])

# create json and write it to an existing empty json file
result = indiv_df_final.to_json(orient="records")
parsed = json.loads(result)
indiv_json = json.dumps(parsed, indent=4) 

with open('../indiv_name_addr.json', 'w') as outfile:
    outfile.write(indiv_json)
    
# import as indiv_unique_name_addr collection

In [6]:
# extracting the values of the indiv_unique_name_address collection
indiv_aql = "FOR x IN indiv_unique_name_address RETURN x"
indiv_query_result = corp_db.AQLQuery(indiv_aql,rawResults=True, batchSize = 1000)
col_indiv = {}
ind_indiv = 0

for indiv in indiv_query_result:
    col_indiv[ind_indiv] = indiv
    ind_indiv += 1

indiv_short_df = pd.DataFrame.from_dict(data = col_indiv, orient = 'index')
indiv_short = indiv_short_df.to_numpy()

In [7]:
print(indiv_ar[0])

['56380105' 'indiv_unique_address/56380105' '_ef6tPHC---' '00i6fl'
 'RAMADAN MOHAMAD ZIAD' 'RAMADAN MOHAMAD'
 '85 NEWTON ST., , NORTHBOROUGH, MA']


In [18]:
def indiv2indiv_same(indiv_ar, indiv_short, batch_size): 
    
    for i in range(0,len(indiv_short)/batch_size):
        print(i)
        print(i*batch_size,(i+1)*batch_size)
        batch_ar = indiv_short[i*batch_size:(i+1)*batch_size][:] 
        
        for j in range(0,batch_ar.shape[0]):
            id1 = batch_ar[j][1]
            indivs = np.where((indiv_ar[:,7]==batch_ar[j][7]))[0]
            id2 = indiv_ar[indivs][:,1]
            for k in id2:
                edge_attributes = {"_from":id1, "_to": k, "type": 'indiv/same/address'}
                edge = corp_db["indiv2indiv"].createDocument(edge_attributes)
                edge.save()

indiv2indiv_same(indiv_ar_2, indiv_short, 1000)

2000
2000000 2001000
2001
2001000 2002000
2002
2002000 2003000
2003
2003000 2004000
2004
2004000 2005000
2005
2005000 2006000
2006
2006000 2007000
2007
2007000 2008000
2008
2008000 2009000
2009
2009000 2010000
2010
2010000 2011000
2011
2011000 2012000
2012
2012000 2013000
2013
2013000 2014000
2014
2014000 2015000
2015
2015000 2016000
2016
2016000 2017000
2017
2017000 2018000
2018
2018000 2019000
2019
2019000 2020000
2020
2020000 2021000
2021
2021000 2022000
2022
2022000 2023000
2023
2023000 2024000
2024
2024000 2025000
2025
2025000 2026000
2026
2026000 2027000
2027
2027000 2028000
2028
2028000 2029000
2029
2029000 2030000
2030
2030000 2031000
2031
2031000 2032000
2032
2032000 2033000
2033
2033000 2034000
2034
2034000 2035000
2035
2035000 2036000
2036
2036000 2037000
2037
2037000 2038000
2038
2038000 2039000
2039
2039000 2040000
2040
2040000 2041000
2041
2041000 2042000
2042
2042000 2043000
2043
2043000 2044000
2044
2044000 2045000
2045
2045000 2046000
2046
2046000 2047000
2047
2047000 